<div style="background:#E9FFF6; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFQ619 - Data Analytics for Strategic Decision Makers (2024)</div>

# IFQ619 :: C1-UnstructuredAnalytics

For this session, the focus will be on analysis of unstructured text. However, the thinking required is similar to approaches to analysing images, video, sound and other unstructured data. Primarily, the analysis is based on the notion that there are useful patterns in the unstructured data which can be obtained mathematically. By converting the data to a mathematical structure, various algorithms can be applied to the structure with the aim of identifying patterns. 

In the case of the `topic modelling` approaches below, many of the techniques are *probabilistic* - that is they mathematically identify the *likelihood* that a feature might be important. Thus, they are never 100% accurate, and their use needs to be mediated by a more pragmatic *useful or not* approach, rather than *right or wrong*.

In [1]:
# Import the necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pandas as pd
import json
import random

### Accessing the data via The Guardian API

See the `Accessing_the_Guardian_API.ipynb` notebook file for details on getting the data. **Note:** This approach may be used for additional data for Assignment 2.

### Read in pre-saved data

To save time, we're loading in pre-saved data that was fetched using the Guardian API.

In [4]:
# Load the data - articles from The Guardian about the war in Ukraine
file_path = "data/"
file_name = "war_articles.json"

with open(f"{file_path}{file_name}",'r', encoding='utf-8') as fp:
    articles = json.load(fp)

print(f"Loaded {len(articles)} articles from {file_name}")

Loaded 864 articles from war_articles.json


Each dictionary entry includes the *title [date]* as `key` and the *body text* from the article as `value`.

In [5]:
article1 = list(articles.items())[0]
print("Key:",article1[0])
print("Value:",article1[1][:300],"...") # Just show first 300 characters

Key: Ukraine war briefing: Trump vows to end war in call with Zelenskiy [2024-07-20T01:34:47Z]
Value:  Donald Trump said he had a “very good call” with Volodymyr Zelenskiy on Friday and pledged to end Ukraine’s war with Russia. The US Republican presidential nominee posted on Truth Social that as president he would “bring peace to the world and end the war that has cost so many lives”. “Both sides w ...


So the values gives us a list of documents that we can analyse.

In [7]:
# Get a list of documents
documents = list(articles.values())

# View first 400 characters of the 1st document
documents[0][:400]

' Donald Trump said he had a “very good call” with Volodymyr Zelenskiy on Friday and pledged to end Ukraine’s war with Russia. The US Republican presidential nominee posted on Truth Social that as president he would “bring peace to the world and end the war that has cost so many lives”. “Both sides will be able to come together and negotiate a deal that ends the violence and paves a path forward to'

### Term Count 

**Finding important terms by the frequency of their occurance**

Using `CountVectorizer` create a `vector` for each document where the dimensionality of the vector is the `vocabulary` (all terms in the collection), and the value of each component is the number of times that the `term` occurs in the document.

All of these analyses, approach the document as a [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. In this approach, the order of the words don't matter. A popular approach that takes into account order is [Word embedding](https://en.wikipedia.org/wiki/Word_embedding). This session does not explore word embedding.

In [8]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
count_vectorizer = CountVectorizer(max_df=0.75,min_df=2,max_features=10000,stop_words="english")
count_dt_matrix = count_vectorizer.fit_transform(articles.values())

In [9]:
# Take a look at the vector for the first document
doc001_vector = count_dt_matrix.toarray()[0]
doc001_vector

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
# Get the 1000 terms identified during the vectorization process
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['00', '000', '000km', ..., 'על', 'של', 'في'], dtype=object)

In [ ]:
# Look at how the counts match up to the terms (for the 1st doc)
doc001_term_counts = list(zip(feature_names,???))
doc001_term_counts

In [12]:
# Take a look at the vocabulary which shows the total counts for whole collection
count_vectorizer.vocabulary_

{'donald': 3039,
 'trump': 9292,
 'good': 4135,
 'volodymyr': 9645,
 'zelenskiy': 9986,
 'friday': 3950,
 'pledged': 6744,
 'end': 3301,
 'ukraine': 9346,
 'russia': 7855,
 'republican': 7582,
 'presidential': 6923,
 'nominee': 6147,
 'posted': 6829,
 'truth': 9296,
 'social': 8350,
 'president': 6922,
 'bring': 1510,
 'peace': 6575,
 'world': 9905,
 'cost': 2377,
 'lives': 5441,
 'sides': 8241,
 'able': 417,
 'come': 2045,
 'negotiate': 6078,
 'deal': 2589,
 'ends': 3314,
 'violence': 9612,
 'path': 6539,
 'forward': 3905,
 'prosperity': 7076,
 'repeatedly': 7552,
 'claimed': 1917,
 'quickly': 7193,
 'offering': 6263,
 'details': 2798,
 'conversation': 2326,
 'congratulated': 2206,
 'securing': 8027,
 'nomination': 6145,
 'condemned': 2170,
 'assassination': 983,
 'attempt': 1032,
 'agreed': 635,
 'arrange': 934,
 'meeting': 5738,
 'person': 6627,
 'discuss': 2923,
 'steps': 8576,
 'make': 5576,
 'fair': 3634,
 'truly': 9291,
 'lasting': 5260,
 'secretary': 8017,
 'state': 8536,
 'ant

#### Display matrix in dataframe

Take the term count matrix and display in a dataframe to make visible the structure


In [13]:
count_dt_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 6, 0, ..., 0, 0, 2],
       ...,
       [0, 6, 0, ..., 0, 0, 0],
       [0, 6, 0, ..., 0, 0, 0],
       [0, 8, 0, ..., 0, 0, 0]])

In [18]:
# Create a new dataframe with the matrix - use titles for the index and terms for the columns
count_df = pd.DataFrame(count_dt_matrix.toarray(), index=articles.keys(), columns=feature_names)
count_df

,00,000,000km,00am,00pm,01am,01pm,02am,02pm,03am,...,zomi,zone,zones,zoo,zuhri,את,ישראל,על,של,في
Ukraine war briefing: Trump vows to end war in call with Zelenskiy [2024-07-20T01:34:47Z],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ukraine war briefing: Turkey launches new Ukrainian warship [2024-08-02T00:18:54Z],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Israel-Gaza war: protesters in Tel Aviv demand end to war – as it happened [2024-05-25T23:15:52Z],0,6,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
Ukraine war briefing: US hits China with sanctions over war supplies to Russia [2024-05-02T00:24:30Z],0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ukraine war briefing: ‘Russia does not control Kursk border’ [2024-08-07T01:23:54Z],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Oliver Dowden reportedly reveals preferred choice for next Tory leader – UK general election as it happened [2024-07-02T19:59:07Z],2,28,0,2,2,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
PM defends response to ICC arrest warrant request – as it happened [2024-05-23T08:26:49Z],0,12,0,1,0,4,1,2,0,2,...,0,0,0,0,0,0,0,0,0,0
Museum loses anti-discrimination case – as it happened [2024-04-09T08:10:14Z],0,6,0,1,0,2,1,2,0,2,...,1,1,0,0,0,0,0,0,0,0
Woman found dead in North Bondi – as it happened [2024-04-30T07:57:49Z],0,6,0,1,0,3,0,1,1,3,...,0,1,0,0,0,0,0,0,0,0


By selecting a row from the dataframe and sorting the values (counts), we can identify the top 10 terms

#### Create a top10 terms dataframe

Using the index from the documents, create a dataframe that can hold the top10 terms for each document. We also include columns for our other analysis (tfidf, lda, nmf)

In [25]:
# Create a dataframe to hold top terms for each analysis type
terms_df = pd.DataFrame(index=count_df.index,columns=['count','tfidf','lda','nmf'])
terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: Trump vows to end war in call with Zelenskiy [2024-07-20T01:34:47Z],NaN,NaN,NaN,NaN
Ukraine war briefing: Turkey launches new Ukrainian warship [2024-08-02T00:18:54Z],NaN,NaN,NaN,NaN
Israel-Gaza war: protesters in Tel Aviv demand end to war – as it happened [2024-05-25T23:15:52Z],NaN,NaN,NaN,NaN
Ukraine war briefing: US hits China with sanctions over war supplies to Russia [2024-05-02T00:24:30Z],NaN,NaN,NaN,NaN
Ukraine war briefing: ‘Russia does not control Kursk border’ [2024-08-07T01:23:54Z],NaN,NaN,NaN,NaN
...,...,...,...,...
Oliver Dowden reportedly reveals preferred choice for next Tory leader – UK general election as it happened [2024-07-02T19:59:07Z],NaN,NaN,NaN,NaN
PM defends response to ICC arrest warrant request – as it happened [2024-05-23T08:26:49Z],NaN,NaN,NaN,NaN
Museum loses anti-discrimination case – as it happened [2024-04-09T08:10:14Z],NaN,NaN,NaN,NaN
Woman found dead in North Bondi – as it happened [2024-04-30T07:57:49Z],NaN,NaN,NaN,NaN


Populate the count column with data created by the count vectorizer.

In [27]:
#For each doc, get the 10 columns with the largest counts
for idx in terms_df.index:
    counts = dict(count_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'count'] = list(counts.keys()) # Just the list of terms

terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: Trump vows to end war in call with Zelenskiy [2024-07-20T01:34:47Z],"[ukraine, friday, zelenskiy, president, minist...",NaN,NaN,NaN
Ukraine war briefing: Turkey launches new Ukrainian warship [2024-08-02T00:18:54Z],"[ukraine, russian, russia, ukrainian, air, tur...",NaN,NaN,NaN
Israel-Gaza war: protesters in Tel Aviv demand end to war – as it happened [2024-05-25T23:15:52Z],"[israel, gaza, israeli, bst, rafah, palestinia...",NaN,NaN,NaN
Ukraine war briefing: US hits China with sanctions over war supplies to Russia [2024-05-02T00:24:30Z],"[russia, russian, ukraine, military, state, we...",NaN,NaN,NaN
Ukraine war briefing: ‘Russia does not control Kursk border’ [2024-08-07T01:23:54Z],"[ukraine, russian, russia, ukrainian, defence,...",NaN,NaN,NaN
...,...,...,...,...
Oliver Dowden reportedly reveals preferred choice for next Tory leader – UK general election as it happened [2024-07-02T19:59:07Z],"[labour, bst, starmer, party, election, says, ...",NaN,NaN,NaN
PM defends response to ICC arrest warrant request – as it happened [2024-05-23T08:26:49Z],"[bst, australia, government, updated, power, e...",NaN,NaN,NaN
Museum loses anti-discrimination case – as it happened [2024-04-09T08:10:14Z],"[bst, updated, defence, australian, australia,...",NaN,NaN,NaN
Woman found dead in North Bondi – as it happened [2024-04-30T07:57:49Z],"[bst, hanson, updated, says, australia, year, ...",NaN,NaN,NaN


In [28]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = count_df.iloc[sample]
    top_terms = dict(count_df.iloc[sample].sort_values(ascending=False).head(10))
    print(f"[{sample}] {doc.name}")
    print("\t- Top terms:",top_terms)

[821] Bob Carr urges New Zealand not to join Aukus – as it happened [2024-04-17T08:12:08Z]
	- Top terms: {'bst': 165, 'government': 81, 'updated': 78, 'defence': 74, 'australia': 56, 'new': 54, 'says': 40, 'national': 38, 'marles': 38, 'minister': 35}
[485] Middle East crisis: threat of Iranian attack on Israel ‘still viable’, says White House – as it happened [2024-04-12T16:48:22Z]
	- Top terms: {'israel': 125, 'israeli': 81, 'gaza': 74, 'iran': 65, 'bst': 42, 'hamas': 39, 'minister': 38, 'uk': 35, 'attack': 33, 'foreign': 32}
[309] ‘Lost for words’: Joe Biden’s tale about cannibals bemuses Papua New Guinea residents [2024-04-19T01:20:56Z]
	- Top terms: {'new': 11, 'guinea': 10, 'biden': 7, 'papua': 7, 'uncle': 5, 'png': 4, 'plane': 4, 'lost': 3, 'aircraft': 3, 'lot': 3}
[272] Morning Mail: Labor regains poll lead, call for better therapy regulation, Netanyahu power grab  [2024-06-17T20:48:13Z]
	- Top terms: {'morning': 5, 'labor': 5, 'australia': 5, 'just': 4, 'target': 4, 'world': 4

### Term Frequency / Inverse Document Frequency (TF/IDF)

**Finding terms that are very common in a document, but less common in the whole collection**

The [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) algorithm takes the term frequencies for a document and divides them by the frequencies of the terms in the whole collection.


In [29]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.75, min_df=2, max_features=10000, stop_words="english"
)

In [31]:
# Get the document vectors
tfidf_dt_matrix = tfidf_vectorizer.fit_transform(articles.values())

# Display the vector for the first document
tfidf_dt_matrix.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

#### Display matrix in dataframe

In [32]:
tfidf_df = pd.DataFrame(tfidf_dt_matrix.toarray(), index=articles.keys(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,00,000,000km,00am,00pm,01am,01pm,02am,02pm,03am,...,zomi,zone,zones,zoo,zuhri,את,ישראל,על,של,في
Ukraine war briefing: Trump vows to end war in call with Zelenskiy [2024-07-20T01:34:47Z],0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Ukraine war briefing: Turkey launches new Ukrainian warship [2024-08-02T00:18:54Z],0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Israel-Gaza war: protesters in Tel Aviv demand end to war – as it happened [2024-05-25T23:15:52Z],0.00000,0.016428,0.0,0.004385,0.005084,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.006089,0.0,0.0,0.0,0.0,0.0,0.0,0.017693
Ukraine war briefing: US hits China with sanctions over war supplies to Russia [2024-05-02T00:24:30Z],0.00000,0.019031,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Ukraine war briefing: ‘Russia does not control Kursk border’ [2024-08-07T01:23:54Z],0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Oliver Dowden reportedly reveals preferred choice for next Tory leader – UK general election as it happened [2024-07-02T19:59:07Z],0.00988,0.046502,0.0,0.005320,0.006168,0.000000,0.000000,0.000000,0.003260,0.002792,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
PM defends response to ICC arrest warrant request – as it happened [2024-05-23T08:26:49Z],0.00000,0.026175,0.0,0.003494,0.000000,0.014864,0.004104,0.007181,0.000000,0.007333,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Museum loses anti-discrimination case – as it happened [2024-04-09T08:10:14Z],0.00000,0.013205,0.0,0.003525,0.000000,0.007498,0.004141,0.007246,0.000000,0.007398,...,0.005205,0.003483,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Woman found dead in North Bondi – as it happened [2024-04-30T07:57:49Z],0.00000,0.010336,0.0,0.002759,0.000000,0.008804,0.000000,0.002836,0.003381,0.008686,...,0.000000,0.002726,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


#### Update the terms matrix

In [33]:
for idx in terms_df.index:
    tfidf = dict(tfidf_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'tfidf'] = list(tfidf.keys()) 

terms_df

,count,tfidf,lda,nmf
Ukraine war briefing: Trump vows to end war in call with Zelenskiy [2024-07-20T01:34:47Z],"[ukraine, friday, zelenskiy, president, minist...","[ukraine, zelenskiy, german, friday, starmer, ...",NaN,NaN
Ukraine war briefing: Turkey launches new Ukrainian warship [2024-08-02T00:18:54Z],"[ukraine, russian, russia, ukrainian, air, tur...","[russian, ukraine, turkey, ukrainian, ivan, go...",NaN,NaN
Israel-Gaza war: protesters in Tel Aviv demand end to war – as it happened [2024-05-25T23:15:52Z],"[israel, gaza, israeli, bst, rafah, palestinia...","[israel, gaza, rafah, icj, bst, israeli, pales...",NaN,NaN
Ukraine war briefing: US hits China with sanctions over war supplies to Russia [2024-05-02T00:24:30Z],"[russia, russian, ukraine, military, state, we...","[russian, russia, ukraine, tanks, chemical, en...",NaN,NaN
Ukraine war briefing: ‘Russia does not control Kursk border’ [2024-08-07T01:23:54Z],"[ukraine, russian, russia, ukrainian, defence,...","[ukraine, russian, russia, isw, ukrainian, def...",NaN,NaN
...,...,...,...,...
Oliver Dowden reportedly reveals preferred choice for next Tory leader – UK general election as it happened [2024-07-02T19:59:07Z],"[labour, bst, starmer, party, election, says, ...","[labour, starmer, bst, postal, sunak, tories, ...",NaN,NaN
PM defends response to ICC arrest warrant request – as it happened [2024-05-23T08:26:49Z],"[bst, australia, government, updated, power, e...","[bst, eraring, updated, australia, origin, nsw...",NaN,NaN
Museum loses anti-discrimination case – as it happened [2024-04-09T08:10:14Z],"[bst, updated, defence, australian, australia,...","[bst, updated, aukus, pillar, ola, japan, tick...",NaN,NaN
Woman found dead in North Bondi – as it happened [2024-04-30T07:57:49Z],"[bst, hanson, updated, says, australia, year, ...","[hanson, bst, bonza, holt, updated, faruqi, qu...",NaN,NaN


#### Compare approaches

In [34]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print()

[711] Morning Mail: Albanese’s defence ploy, Modi may lose majority in surprise result, Djokovic down and out in Paris [2024-06-04T20:48:03Z]
	- Counts:	 ['new', 'minister', 'prime', 'news', 'morning', 'defence', 'election', 'world', 'company', 'open']
	- TFIDF:	 ['farage', 'modi', 'weight', 'covert', 'new', 'newborn', 'nigel', 'dreams', 'manchester', 'narendra']

[649] Morning Mail: Dutton goes nuclear, $100bn ‘crisis profits’, Tories face wipeout [2024-06-18T20:45:45Z]
	- Counts:	 ['morning', 'state', 'australia', 'queensland', 'nuclear', 'profits', 'new', 'world', 'according', 'australian']
	- TFIDF:	 ['profits', 'loophole', 'deepfakes', 'queensland', 'nuclear', 'morning', 'australia', 'state', 'massacre', 'crossword']

[803] Wong warns Australians in Lebanon to ‘leave immediately’ – as it happened [2024-08-04T06:09:24Z]
	- Counts:	 ['bst', 'government', 'updated', 'albanese', 'gambling', '12', 'says', 'australia', 'indigenous', 'minister']
	- TFIDF:	 ['bst', 'gambling', 'updated', 

### Topic modelling with Latent Dirichlet Allocation (LDA)

[LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is an algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It is a generative probabilistic approach to *decomposition* of the document-term matrix into 2 factor matrices: document-topic and topic-term.

![img](https://editor.analyticsvidhya.com/uploads/26864dtm.JPG)

*Source: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/06/part-2-topic-modeling-and-latent-dirichlet-allocation-lda-using-gensim-and-sklearn/)*

The LDA model requires the number of topics to be set in advance. As it is a generative model, it also runs over a number of iterations. These values usually need to be experimented with to obtain quality topics.

In [37]:
# Set number of topics
num_topics = 20
# Set max number of iteractions
max_iterations = 30

# Create the model
lda_model = LatentDirichletAllocation(n_components=num_topics,max_iter=max_iterations,learning_method='online')

# Fit the model to the data, and use the model to transform the data (do the decomposition)
doc_topic_matrix = lda_model.fit_transform(count_dt_matrix) # Note that this uses the counts already processed

# Obtain the topics
topic_term_matrix = lda_model.components_

#### View the topics

In [39]:
# Get the topics and their terms
lda_topic_dict = {}
for index, topic in enumerate(topic_term_matrix):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    lda_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in lda_topic_dict.items():
    print(k)
    print(v)
    print()

{'university': 1362.0347489614792, 'students': 820.530748016376, 'police': 778.4408932048277, 'palestine': 583.774555420783, 'protesters': 546.6122007210696, 'pro': 478.2289645937253, 'israel': 414.3666915495476, 'new': 411.14015905339886, 'bst': 390.5589150429984, 'campus': 375.74720265498945}
{'israel': 7947.976993759037, 'gaza': 7219.440289620926, 'israeli': 5982.888225035548, 'bst': 4055.3551833861598, 'hamas': 3335.565526255359, 'killed': 2530.245274064308, 'military': 2469.067921089506, 'palestinian': 2429.6001813800654, 'minister': 1925.2112842757606, 'aid': 1811.0671270195862}
{'gaza': 0.055194626731996876, 'israel': 0.05462780558280499, 'israeli': 0.053987149237401895, 'bst': 0.05332912888144294, 'military': 0.051819424436908144, 'killed': 0.05174896723308683, 'minister': 0.05173923934755056, 'hamas': 0.05163379465743726, 'palestinian': 0.051575117303519415, 'ukraine': 0.05115296174251587}
{'bst': 0.05247009302832329, 'ukraine': 0.0517243021828818, 'russian': 0.051676157707867

#### List of topics for each document

In [40]:
doc_topic_matrix[0]

array([1.48367957e-04, 8.91830461e-02, 1.48367953e-04, 1.48367953e-04,
       1.48367953e-04, 1.48367953e-04, 1.94517764e-01, 1.48367953e-04,
       1.48367953e-04, 1.48367953e-04, 1.48367953e-04, 1.48367956e-04,
       1.48367954e-04, 1.48367953e-04, 1.48367956e-04, 1.48367953e-04,
       1.48367953e-04, 1.86175029e-01, 1.48367953e-04, 5.27750274e-01])

In [41]:
doc_topic_matrix[200]

array([1.38657796e-05, 9.81359988e-01, 1.38657793e-05, 1.38657793e-05,
       1.38657793e-05, 1.38657793e-05, 1.74792527e-02, 1.38657793e-05,
       1.38657793e-05, 1.38657793e-05, 1.38657793e-05, 1.38657795e-05,
       1.38657794e-05, 2.75918392e-04, 1.38657796e-05, 1.38657793e-05,
       1.38657793e-05, 6.62988107e-04, 1.38657793e-05, 1.38657795e-05])

#### Update the terms matrix

In [54]:
for idx,topic in enumerate(doc_topic_matrix):
     # enumerate just takes a list and returns the value and a counter for the loop starting from 0
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['lda'].iloc[idx] = list(top_topic.keys())

terms_df

KeyError: 'topic_9346'

#### Compare approaches

In [45]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print()

[19] In Bathurst, memories of dispossession and war simmer beneath the surface [2024-08-14T00:00:27Z]
	- Counts:	 ['wiradjuri', 'bathurst', 'white', 'police', 'british', 'law', 'plains', 'martial', 'says', 'brisbane']
	- TFIDF:	 ['wiradjuri', 'bathurst', 'plains', 'martial', 'stewart', 'dispossession', 'mounted', 'colonial', 'resistance', 'white']
	- LDA:		 ['history', 'indigenous', 'aboriginal', 'university', 'research', 'says', 'ocean', 'food', 'eugenics', 'wagga']

[37] Ukraine war briefing: Backlash at freed Russians calling for eased sanctions and negotiation [2024-08-06T00:32:13Z]
	- Counts:	 ['russian', 'ukraine', 'ukrainian', 'russia', 'russians', 'military', 'sanctions', 'putin', 'kyiv', 'reported']
	- TFIDF:	 ['russian', 'russians', 'ukrainian', 'ukraine', 'putin', 'sanctions', 'kara', 'murza', 'russia', 'kyiv']
	- LDA:		 ['ukraine', 'russian', 'russia', 'ukrainian', 'region', 'military', 'defence', 'forces', 'zelenskiy', 'attack']

[117] Ukraine war briefing: Zelenskiy ackno

### Topic modelling with Non-negative Matrix Factorisation (NMF)


[NMF](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is a different algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It also factorises the document-term matrix into 2 factor matrices: document-topic and topic-term.

In [47]:
# Set the number of topics
num_topics = 20

# Create the model
nmf_model = NMF(n_components=num_topics,init='random',beta_loss='frobenius')

# Fit the model to the data and use it to transform the data
doc_topic_nmf = nmf_model.fit_transform(tfidf_dt_matrix)

topic_term_nmf = nmf_model.components_

In [48]:
# Get the topics and their terms
nmf_topic_dict = {}
for index, topic in enumerate(topic_term_nmf):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    #print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    nmf_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in nmf_topic_dict.items():
    print(k)
    print(v)
    print()

topic_0
{'bst': 3.0999, 'updated': 1.4364, 'government': 0.5073, 'says': 0.3535, 'nsw': 0.3533, 'australia': 0.3476, 'police': 0.3452, 'minister': 0.339, '12': 0.3204, '11': 0.3158}

topic_1
{'dutton': 0.5564, 'visas': 0.4911, 'visa': 0.4583, 'gaza': 0.3872, 'australia': 0.3644, 'asio': 0.2548, 'government': 0.2235, 'palestinians': 0.2142, 'security': 0.2095, 'steggall': 0.2072}

topic_2
{'ukraine': 2.6958, 'russian': 2.2498, 'russia': 1.7448, 'ukrainian': 1.4785, 'zelenskiy': 0.7117, 'region': 0.7053, 'kyiv': 0.556, 'kharkiv': 0.5369, 'moscow': 0.4974, 'military': 0.4856}

topic_3
{'israeli': 1.1447, 'gaza': 1.1093, 'israel': 0.9981, 'bst': 0.6119, 'hamas': 0.5663, 'killed': 0.4623, 'lebanon': 0.4373, 'al': 0.3846, 'palestinian': 0.3714, 'hezbollah': 0.3613}

topic_4
{'harris': 4.9181, 'trump': 3.8945, 'walz': 2.5251, 'kamala': 2.2496, 'biden': 1.7624, 'vance': 1.4239, 'president': 1.3862, 'presidential': 1.3777, 'donald': 1.308, 'democratic': 1.2646}

topic_5
{'labour': 1.1742, 'star

#### Update the terms matrix

In [52]:
for idx,topic in enumerate(doc_topic_nmf):
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['nmf'].iloc[idx] = list(top_topic.keys())

terms_df

/tmp/ipykernel_100/2376733025.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  terms_df['nmf'].iloc[idx] = list(top_topic.keys())


,count,tfidf,lda,nmf
Ukraine war briefing: Trump vows to end war in call with Zelenskiy [2024-07-20T01:34:47Z],"[ukraine, friday, zelenskiy, president, minist...","[ukraine, zelenskiy, german, friday, starmer, ...","[ukraine, russian, russia, ukrainian, region, ...","[gaza, israel, israeli, bst, military, killed,..."
Ukraine war briefing: Turkey launches new Ukrainian warship [2024-08-02T00:18:54Z],"[ukraine, russian, russia, ukrainian, air, tur...","[russian, ukraine, turkey, ukrainian, ivan, go...","[ukraine, russian, russia, ukrainian, region, ...","[gaza, israel, israeli, bst, military, killed,..."
Israel-Gaza war: protesters in Tel Aviv demand end to war – as it happened [2024-05-25T23:15:52Z],"[israel, gaza, israeli, bst, rafah, palestinia...","[israel, gaza, rafah, icj, bst, israeli, pales...","[israel, gaza, israeli, bst, hamas, killed, mi...","[israel, bst, gaza, ukraine, israeli, russian,..."
Ukraine war briefing: US hits China with sanctions over war supplies to Russia [2024-05-02T00:24:30Z],"[russia, russian, ukraine, military, state, we...","[russian, russia, ukraine, tanks, chemical, en...","[ukraine, russian, russia, ukrainian, region, ...","[gaza, israel, israeli, bst, military, killed,..."
Ukraine war briefing: ‘Russia does not control Kursk border’ [2024-08-07T01:23:54Z],"[ukraine, russian, russia, ukrainian, defence,...","[ukraine, russian, russia, isw, ukrainian, def...","[ukraine, russian, russia, ukrainian, region, ...","[gaza, israel, israeli, bst, military, killed,..."
...,...,...,...,...
Oliver Dowden reportedly reveals preferred choice for next Tory leader – UK general election as it happened [2024-07-02T19:59:07Z],"[labour, bst, starmer, party, election, says, ...","[labour, starmer, bst, postal, sunak, tories, ...","[election, party, labour, bst, starmer, sunak,...","[goddess, practitioners, gaza, israel, israeli..."
PM defends response to ICC arrest warrant request – as it happened [2024-05-23T08:26:49Z],"[bst, australia, government, updated, power, e...","[bst, eraring, updated, australia, origin, nsw...","[bst, updated, government, australia, minister...","[university, students, police, palestine, prot..."
Museum loses anti-discrimination case – as it happened [2024-04-09T08:10:14Z],"[bst, updated, defence, australian, australia,...","[bst, updated, aukus, pillar, ola, japan, tick...","[bst, updated, government, australia, minister...","[university, students, police, palestine, prot..."
Woman found dead in North Bondi – as it happened [2024-04-30T07:57:49Z],"[bst, hanson, updated, says, australia, year, ...","[hanson, bst, bonza, holt, updated, faruqi, qu...","[bst, updated, government, australia, minister...","[university, students, police, palestine, prot..."


### Compare approaches

In [53]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print("\t- NMF:\t\t",doc['nmf'])
    print()

[97] Ukraine war briefing: Russia launches ‘double tap’ missile attack on a small town near Kharkiv [2024-07-14T04:10:23Z]
	- Counts:	 ['ukraine', 'russian', 'europe', 'missiles', 'european', 'support', 'zelenskiy', 'peskov', 'reports', 'defence']
	- TFIDF:	 ['ukraine', 'peskov', 'europe', 'european', 'russian', 'missiles', 'kremlin', 'zelenskiy', 'germany', 'capitals']
	- LDA:		 ['ukraine', 'russian', 'russia', 'ukrainian', 'region', 'military', 'defence', 'forces', 'zelenskiy', 'attack']
	- NMF:		 ['gaza', 'israel', 'israeli', 'bst', 'military', 'killed', 'minister', 'hamas', 'palestinian', 'ukraine']

[101] Ukraine war briefing: Russian forces launch deadly attack on Kharkiv lakeside resort [2024-05-20T00:47:28Z]
	- Counts:	 ['russian', 'ukraine', 'kharkiv', 'town', 'villages', 'forces', 'region', 'russia', 'gladkov', 'foreign']
	- TFIDF:	 ['russian', 'villages', 'gladkov', 'kharkiv', 'ukraine', 'shelled', 'town', 'resort', 'injuring', 'ukrainian']
	- LDA:		 ['ukraine', 'russian', '

Sometimes different approaches are used. Sometimes LDA is better, sometimes nmf. LDA/nmf use different. How do we evaluate which is best?

Andrew recommendsnot just using counts, but it can do pretty good sometimes